In [1]:
import numpy as np
import pandas as pd
import orangecontrib.associate.fpgrowth as oaf
import datetime
import argparse
#parser = argparse.ArgumentParser()
#parser.add_argument('--train_dir', type=str, default='./trainDataGZ.txt')
#parser.add_argument('--rules_dir', type=str, default='./rulesDataGZ.txt')
#parser.add_argument('--support', type=float, default=0.05)
#parser.add_argument('--confidence',type=float,default=0.2)
#args = parser.parse_args()

In [2]:
y=np.random.randint(0,2,1000)
hum=np.random.randint(0,101,[1000,])
tem=np.random.uniform(-50.0,40.0,[1000,])
is_gas=np.random.randint(0,2,1000)
per_con=np.random.randint(0,101,[1000,])
is_metal=np.random.randint(0,2,1000)
d_age=np.random.randint(1,11,1000)
d_type=np.random.randint(1,4,1000)

In [3]:
df=pd.DataFrame({
    "hum":hum,
    "tem":tem,
    "is_gas":is_gas,
    "per_con":per_con,
    "is_metal":is_metal,
    "d_age":d_age,
    "d_type":d_type,
    'is_fault':y
})

In [4]:
#df.to_csv('trainDataGZ.csv',index=False)

In [5]:
def data_pre(path):
    df=pd.read_csv(path)
    #iscretization
    bins=[0,50,70,90,100]
    df['hum']=pd.cut(df['hum'],bins,right=False,labels=['1','2','3','4'])
    bins=[-50.0,-20.0,-10.0,0,10.0,20.0,30.0,41.0]
    df['tem']=pd.cut(df['tem'],bins,right=False,labels=['1','2','3','4','5','6','7'])
    bins=[0,20,100]
    df['per_con']=pd.cut(df['per_con'],bins,right=False,labels=['1','2'])
    bins=[0,2,5,11]
    df['d_age']=pd.cut(df['d_age'],bins,right=False,labels=['1','2','3'])
    #type trans
    df['is_gas']=df['is_gas'].apply(str)
    df['is_metal']=df['is_metal'].apply(str)
    df['d_type']=df['d_type'].apply(str)
    df['is_fault']=df['is_fault'].apply(str)
    #one_hot
    df=pd.get_dummies(df)
    strDecode=df.columns.values.tolist()
    data=df.values
    data.dtype=bool
    return data,strDecode

In [6]:
def model(data,support=0.05,confidence=0.2):
    fre_ite=dict(oaf.frequent_itemsets(data,support))  #这里设置置信度
    rules = oaf.association_rules(fre_ite,confidence)
    result=list(rules)
    return result

In [7]:
def dealResult(rules,strDecode):
    returnRules = []
    for i in rules:
        j= list(i[1])
        if len(j)==1 and strDecode[j[0]].startswith('is_fault') :
            temStr = [];
            temp=''
            for k in i[0]:
                temp = temp+strDecode[k]+'&'
            temp = temp[:-1]
            temp = temp + ' ==> '
            temp = temp +strDecode[j[0]] 
            temStr.append(temp)
            temStr.append(i[2])
            temStr.append(i[3])
            returnRules.append(temStr)
    returnRules=list(sorted(returnRules,key=lambda x:(-x[2],-x[1])))
    return returnRules
    


In [8]:
support=0.05
confidence=0.2
train_dir='trainDataGZ.csv'
rules_dir='rulesDataGZ.txt'
data,strDecode=data_pre(train_dir)
result=model(data,support,confidence)
printRules= dealResult(result,strDecode)
file=open(rules_dir,'w')
print('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
file.write('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
for line in printRules:
    s='\t'.join([str(i) for i in line])
    print(s+'\n')
    file.write(s+'\n')
file.close()
    

Rules	support	confidence

tem_3 ==> is_fault_0	64	0.6666666666666666

tem_3&per_con_2 ==> is_fault_0	51	0.6538461538461539

hum_1&d_type_1&is_gas_1&per_con_2 ==> is_fault_0	51	0.6296296296296297

is_metal_1&is_gas_0&d_type_3 ==> is_fault_1	51	0.6144578313253012

hum_1&d_type_1&is_metal_0 ==> is_fault_0	50	0.6097560975609756

d_age_2&d_type_3 ==> is_fault_1	51	0.6

hum_1&is_gas_1&per_con_2&is_metal_0 ==> is_fault_0	56	0.5957446808510638

hum_1&d_type_1&is_gas_1 ==> is_fault_0	56	0.5957446808510638

d_age_2&d_type_2 ==> is_fault_0	62	0.5904761904761905

hum_1&is_gas_1&is_metal_0 ==> is_fault_0	66	0.5892857142857143

is_gas_0&d_type_3&per_con_2 ==> is_fault_1	79	0.5851851851851851

is_metal_1&hum_3&per_con_2 ==> is_fault_1	50	0.5813953488372093

is_gas_0&d_type_3 ==> is_fault_1	91	0.5796178343949044

hum_1&d_type_2&per_con_2 ==> is_fault_0	74	0.578125

hum_1&is_gas_0&d_type_3 ==> is_fault_1	52	0.5777777777777777

hum_1&is_gas_1&per_con_2 ==> is_fault_0	118	0.5756097560975609

hum_1&is_met